In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import cv2
import os
import tifffile as tiff

# Load your image and ground truth mask

def load_images_from_folder(folder, target_size=(256, 256)):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = tiff.imread(img_path)  
        if img is not None:
            img = cv2.resize(img, target_size)  # Resize image
            img = img / np.max(img) #  255.0  # Normalize to [0, 1]
            img = np.expand_dims(img, axis=-1)  # Add channel dimension
            images.append(img)
    return np.array(images)

# Load images
X_train = load_images_from_folder(r"C:\Users\colin\TrainingImages")

def load_masks_from_folder(folder, target_size=(256, 256)):
    masks = []
    for filename in os.listdir(folder):
        mask_path = os.path.join(folder, filename)
        mask = tiff.imread(mask_path)  # Read as grayscale
        if mask is not None:
            mask = cv2.resize(mask, target_size)  # Resize mask
            mask = mask / np.max(mask)  # Normalize to [0, 1]
            mask = np.expand_dims(mask, axis=-1)  # Add channel dimension
            masks.append(mask)
    return np.array(masks)

# Load masks
Y_train = load_masks_from_folder(r"C:\Users\colin\TrainingMasks")



In [3]:
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
Y_train = (Y_train > 0).astype(np.uint8)


X_train shape: (600, 256, 256, 1)
Y_train shape: (600, 256, 256, 1)


In [4]:
def unet_model(input_size=(256, 256, 1)):
    inputs = layers.Input(input_size)
    
    # Down-sampling path
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)
    
    conv5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(conv5)
    
    # Up-sampling path
    up6 = layers.Conv2D(512, (2, 2), activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv5))
    merge6 = layers.concatenate([conv4, up6], axis=3)
    conv6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(merge6)
    conv6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv6)
    
    up7 = layers.Conv2D(256, (2, 2), activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv6))
    merge7 = layers.concatenate([conv3, up7], axis=3)
    conv7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(merge7)
    conv7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)
    
    up8 = layers.Conv2D(128, (2, 2), activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv7))
    merge8 = layers.concatenate([conv2, up8], axis=3)
    conv8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(merge8)
    conv8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv8)
    
    up9 = layers.Conv2D(64, (2, 2), activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv8))
    merge9 = layers.concatenate([conv1, up9], axis=3)
    conv9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(merge9)
    conv9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)
    conv9 = layers.Conv2D(2, (3, 3), activation='relu', padding='same')(conv9)
    conv10 = layers.Conv2D(1, (1, 1), activation='sigmoid')(conv9)
    
    model = models.Model(inputs, conv10)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model


In [5]:
# Assuming you have prepared your training data
# X_train = np.array([image])  # Replace with your actual training images
# Y_train = np.array([mask])   # Replace with your actual masks

model = unet_model(input_size=(256, 256, 1))
model.summary()

model.fit(X_train, Y_train, batch_size=1, epochs=10)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 256, 256, 1)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 256, 256, 64)      │             640 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 256, 256, 64)      │          36,928 │ conv2d[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 128, 128, 64)      │               0 │ conv2d_1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 128, 128, 128)     │          73,856 │ max_pooling2d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 128, 128, 128)     │         147,584 │ conv2d_2[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_1               │ (None, 64, 64, 128)       │               0 │ conv2d_3[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_4 (Conv2D)             │ (None, 64, 64, 256)       │         295,168 │ max_pooling2d_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_5 (Conv2D)             │ (None, 64, 64, 256)       │         590,080 │ conv2d_4[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_2               │ (None, 32, 32, 256)       │               0 │ conv2d_5[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_6 (Conv2D)             │ (None, 32, 32, 512)       │       1,180,160 │ max_pooling2d_2[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_7 (Conv2D)             │ (None, 32, 32, 512)       │       2,359,808 │ conv2d_6[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_3               │ (None, 16, 16, 512)       │               0 │ conv2d_7[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_8 (Conv2D)             │ (None, 16, 16, 1024)      │       4,719,616 │ max_pooling2d_3[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_9 (Conv2D)             │ (None, 16, 16, 1024)      │       9,438,208 │ conv2d_8[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ up_sampling2d (UpSampling2D)  │ (None, 32, 32, 1024)      │               

 Total params: 31,031,685 (118.38 MB)

 Trainable params: 31,031,685 (118.38 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 419s 693ms/step - accuracy: 0.9089 - loss: 0.6362
Epoch 2/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 411s 685ms/step - accuracy: 0.9254 - loss: 0.4662
Epoch 3/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 422s 703ms/step - accuracy: 0.9271 - loss: 0.3692
Epoch 4/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 416s 693ms/step - accuracy: 0.9256 - loss: 0.3190
Epoch 5/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 414s 690ms/step - accuracy: 0.9245 - loss: 0.2930
Epoch 6/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 413s 688ms/step - accuracy: 0.9281 - loss: 0.2718
Epoch 7/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 415s 692ms/step - accuracy: 0.9302 - loss: 0.2599
Epoch 8/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 410s 683ms/step - accuracy: 0.9292 - loss: 0.2585
Epoch 9/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 411s 685ms/step - accuracy: 0.9240 - loss: 0.2692
Epoch 10/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 412s 686ms/step - accuracy: 0.9286 - loss: 0.2576


In [6]:
# Save the entire model to a keras file
model.save(r"C:\Users\colin\my_model.keras")


In [7]:
import cv2

# Load and preprocess a new image
new_image = tiff.imread(r"C:\Users\colin\ModelTest.tif")
print("\n2D array shape:", new_image.shape)

new_image = new_image / np.max(new_image)
new_image = np.expand_dims(new_image, axis=-1)

# Predict the mask
predicted_mask = model.predict(np.expand_dims(new_image, axis=0))

# print(predicted_mask)

# Convert to the range [0, 65535]
predicted_mask = (predicted_mask * 65535).astype(np.uint16)

print(predicted_mask)

# Convert to unsigned int 16 for SB
final_image_uint16 = predicted_mask.astype(np.uint16)

# Save the result to a 16-bit TIFF file
file_path = 'TF_Prediction.tif'
tiff.imwrite(file_path, final_image_uint16)

print(f"Predicted Mask saved to {file_path}")

# Post-process and visualize the result
#predicted_mask = (predicted_mask[0, :, :, 0] > 0.5).astype(np.uint8) * 255
#cv2.imshow('Predicted Mask', predicted_mask)
#cv2.waitKey(0)
#cv2.destroyAllWindows()



2D array shape: (256, 256)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 507ms/step
[[[[4990]
   [4990]
   [4990]
   ...
   [4990]
   [4990]
   [4990]]

  [[4990]
   [4990]
   [4990]
   ...
   [4990]
   [4990]
   [4990]]

  [[4990]
   [4990]
   [4990]
   ...
   [4990]
   [4990]
   [4990]]

  ...

  [[4990]
   [4990]
   [4990]
   ...
   [4990]
   [4990]
   [4990]]

  [[4990]
   [4990]
   [4990]
   ...
   [4990]
   [4990]
   [4990]]

  [[4990]
   [4990]
   [4990]
   ...
   [4990]
   [4990]
   [4990]]]]
Predicted Mask saved to TF_Prediction.tif
